In [3]:
"""
This script reads a directory of bscscan exports and calculates the total gas fees paid
Useful for taxes if you haen't been keeping track during the year
"""

import sys
import pandas as pd
import os
import glob
import numpy as np
import re

# path = sys.stdin.read().strip()
# path = '/Volumes/Finance_Vault/Mom_2024/Crypto_fees'  # Enter your path here
path = '/Volumes/Finance_Vault/Taxes_2024/BSC_Data'  # Enter your path here
csv_files = glob.glob(os.path.join(path, "export*.csv"))
fees = {}
for f in csv_files:
    wallet = re.search(r"-(.*)\.csv", f).group(1)
    df = pd.read_csv(f, index_col=False)
    if 'TxnFee(BNB)' in df.columns:
        df['fee_usd'] = np.multiply(df['TxnFee(BNB)'], df['Historical $Price/BNB'])
        df['fee_native'] = df['TxnFee(BNB)']
        df['token'] = 'BNB'
    elif 'TxnFee(ETH)' in df.columns:
        df['fee_usd'] = np.multiply(df['TxnFee(ETH)'], df['Historical $Price/ETH'])
        df['fee_native'] = df['TxnFee(ETH)']
        df['token'] = 'ETH'
    else:
        print(f"No fee column found in {f}")
        continue
    fees[wallet] = {'USD': np.sum(df['fee_usd']), 
                    'Native': np.sum(df['fee_native']),
                    'Token:': df['token'][0]
                    }
    
total_fees_by_token = {}
for wallet, wallet_data in fees.items():
    print(f"Processing wallet: {wallet}")
    print(f"Wallet data: {wallet_data}")

    token = wallet_data['Token:']

    if token not in total_fees_by_token:
        total_fees_by_token[token] = {
            'USD': 0,
            'Native': 0
        }

    total_fees_by_token[token]['USD'] += wallet_data['USD']
    total_fees_by_token[token]['Native'] += wallet_data['Native']


Processing wallet: 0x00d0f8fbf72d67821984f172bc19c3904467950f
Wallet data: {'USD': np.float64(216.44526583843435), 'Native': np.float64(0.33853293711293997), 'Token:': 'BNB'}
Processing wallet: 0x8f9acea6bc63470aa83169637cbf132482a28d55 (1)
Wallet data: {'USD': np.float64(68.12363850643241), 'Native': np.float64(0.0201731222727691), 'Token:': 'ETH'}
Processing wallet: 0x7ef564536bd8d4887fefda9a1c136e92d7696f2f
Wallet data: {'USD': np.float64(2.3019985736789983), 'Native': np.float64(0.0007254789549544001), 'Token:': 'ETH'}
Processing wallet: 0xd8091c5092f1345382c487015a939ccd339a9cb8
Wallet data: {'USD': np.float64(93.27607424932802), 'Native': np.float64(0.15030214779999998), 'Token:': 'BNB'}
Processing wallet: 0x8f9acea6bc63470aa83169637cbf132482a28d55
Wallet data: {'USD': np.float64(124.04789213770864), 'Native': np.float64(0.22588692712587996), 'Token:': 'BNB'}


In [4]:
print("Total fees by token:")
for token, fees in total_fees_by_token.items():
    print(f"Token: {token}")
    print(f"USD fees: {np.round(fees['USD'], 2)}")
    print(f"Native fees: {np.round(fees['Native'], 6)}")
    print("-" * 30)

Total fees by token:
Token: BNB
USD fees: 433.77
Native fees: 0.714722
------------------------------
Token: ETH
USD fees: 70.43
Native fees: 0.020899
------------------------------
